In [38]:
import boto3
from astropy.io import fits

s3 = boto3.client("s3", endpoint_url="https://s3dfrgw.slac.stanford.edu")
o=s3.list_objects(Bucket="rubin-sts", Prefix="LSSTCam/20231125/MC_C_20231125_000312/MC_C_20231125_000312_R44")
o

{'ResponseMetadata': {'RequestId': 'tx0000008f1823eab75ec63-0065623d6c-195bfd-default',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'transfer-encoding': 'chunked',
   'x-amz-request-id': 'tx0000008f1823eab75ec63-0065623d6c-195bfd-default',
   'content-type': 'application/xml',
   'date': 'Sat, 25 Nov 2023 18:31:08 GMT'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Marker': '',
 'Contents': [{'Key': 'LSSTCam/20231125/MC_C_20231125_000312/MC_C_20231125_000312_R44_SG0.fits',
   'LastModified': datetime.datetime(2023, 11, 25, 18, 16, 27, 125000, tzinfo=tzlocal()),
   'ETag': '"13e172fae4b301a3ab6fe2414842c0a7"',
   'Size': 25260480,
   'StorageClass': 'STANDARD',
   'Owner': {'DisplayName': 'Rubin Test Stand', 'ID': 'rubin-sts'}},
  {'Key': 'LSSTCam/20231125/MC_C_20231125_000312/MC_C_20231125_000312_R44_SG0.json',
   'LastModified': datetime.datetime(2023, 11, 25, 18, 16, 24, 194000, tzinfo=tzlocal()),
   'ETag': '"cf17c2b448dc460abf1aab2407721485"',
   'Size': 3058,
   

In [39]:
obj=s3.get_object(Bucket='rubin-sts',Key=o['Contents'][0]['Key'])

fsspec_kwargs = {
    "default_block_size": 100000,
    "default_cache_type": "bytes",
    "endpoint_url": "https://s3dfrgw.slac.stanford.edu"
}

fh=fits.open(f"s3://rubin-sts/{o['Contents'][0]['Key']}",use_fsspec=True,fsspec_kwargs = fsspec_kwargs)
fh[0].header

SIMPLE  =                    T / Java FITS: Fri Nov 24 17:04:46 PST 2023        
BITPIX  =                    8 / bits per data value                            
NAXIS   =                    0 / number of axes                                 
EXTEND  =                    T / Extensions are permitted                       
DATE    = '2023-11-25T18:16:57.311' / Creation Date and Time of File            
MJD     =   60273.761774432845 / Modified Julian Date that the file was written 
RUNNUM  = '13593   '           / The Run Number                                 
CCD_MANU= 'ITL     '           / CCD Manufacturer                               
CCD_TYPE= '3800C   '           / CCD Model Number                               
TESTTYPE= 'FLAT' / BIAS, DARK, FE55, FLAT, LAMBDA, PERSISTENCE, SPOT, SFLAT_<lam
IMGTYPE = 'FLAT    '           / BIAS, DARK, FE55, FLAT, FLAT_<lam>, SPOT, PPUMP
DETSIZE = '[1:4072,1:4000]'                                                     
EXPTIME =                 15

In [57]:
import lsst.afw.display as afwDisplay

def display(exposure,title=None,frame=0):
    afw_display = afwDisplay.Display(frame=frame, backend='matplotlib')
    afw_display.scale('asinh', 'zscale')
    afw_display.setMaskTransparency(90)
    afw_display.mtv(exposure.getMaskedImage())
    
    maskPlaneDict = exposure.getMask().getMaskPlaneDict()
    maskKeyColorList = []
    for k in maskPlaneDict.keys():
        if afw_display.getMaskPlaneColor(k) is not None:
            maskKeyColorList.append(k + ":" + afw_display.getMaskPlaneColor(k))
    print(maskKeyColorList)
    plt.title(title)


import matplotlib.pyplot as plt

from astropy.wcs import WCS
from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename

display(fh)

AttributeError: 'HDUList' object has no attribute 'getMaskedImage'

<Figure size 640x480 with 0 Axes>

In [13]:
from lsst.daf.butler import Butler

repo_path = "/repo/ir2"
butler = Butler(repo_path,collections=['LSSTCam/photodiode','LSSTCam/raw/all'],instrument='LSSTCam')
registry = butler.registry


498

In [22]:
for runnum in [
    '13537',
    '13571',
    '13573',
    '13575',
    '13518',
    '13519',
    '13520',
    '13521',
    '13593',
    '13594'
]:
    print(runnum, len(list(registry.queryDimensionRecords('exposure',where=f'exposure.science_program=\'{runnum:s}\''))))

13537 1242
13571 1264
13573 1242
13575 71
13518 31
13519 336
13520 257
13521 320
13593 498
13594 1824


In [44]:
print(list(registry.queryDimensionRecords('exposure',where=f'exposure.science_program=\'{runnum:s}\' and exposure.obs_id=\'{obs_id}\'' )))


[exposure.RecordClass(instrument='LSSTCam', id=2023111902385, physical_filter='ef_43', obs_id='MC_C_20231119_002385', exposure_time=15.0, dark_time=15.1038, observation_type='flat', observation_reason='flat', day_obs=20231119, seq_num=2385, seq_start=2385, seq_end=2385, group_name='2023111902385', group_id=2023111902385, target_name='UNKNOWN', science_program='13573', tracking_ra=None, tracking_dec=None, sky_angle=None, azimuth=None, zenith_angle=None, has_simulated=False, timespan=Timespan(begin=astropy.time.Time('2023-11-20 01:33:18.183998', scale='tai', format='iso'), end=astropy.time.Time('2023-11-20 01:33:33.287000', scale='tai', format='iso')))]


In [76]:
for runnum in [
    '13537',
    '13571',
    '13573',
    '13575',
    '13518',
    '13519',
    '13520',
    '13521',
    '13593',
    '13594'
]:
    obs_ids = list(set([ x.obs_id for x in list(registry.queryDimensionRecords('exposure',where=f'exposure.science_program=\'{runnum:s}\'')) ]))
    sorted(obs_ids)
    for obs_id in sorted(obs_ids):
        n=len(list(registry.queryDatasets('raw',where=f'exposure.science_program=\'{runnum:s}\' and exposure.obs_id=\'{obs_id}\'')))
        if n==205:
            continue
        print(obs_id, n)

MC_C_20231119_000672 54
MC_C_20231119_000673 166
MC_C_20231119_000674 124
MC_C_20231119_000676 124
MC_C_20231119_000677 151
MC_C_20231119_000683 178
MC_C_20231119_001254 151
MC_C_20231119_001255 202
MC_C_20231119_001257 178
MC_C_20231119_001258 178
MC_C_20231119_001259 178
MC_C_20231107_000167 103
MC_C_20231107_000459 204
MC_C_20231107_000669 203
MC_C_20231107_000670 204
MC_C_20231107_000671 203
MC_C_20231107_000674 202
MC_C_20231107_000675 202
MC_C_20231107_000676 204
MC_C_20231107_000677 202
MC_C_20231107_000678 177
MC_C_20231107_000679 177
MC_C_20231107_000680 177
MC_C_20231107_000681 204
MC_C_20231107_000700 178
MC_C_20231108_000022 199
MC_C_20231108_000053 178
MC_C_20231108_000103 178
MC_C_20231108_000104 178
MC_C_20231108_000119 184
MC_C_20231108_000120 199
MC_C_20231108_000124 178
MC_C_20231108_000154 178
MC_C_20231108_000171 178
MC_C_20231108_000184 178
MC_C_20231108_000311 204
MC_C_20231108_000313 204
MC_C_20231108_000315 178
MC_C_20231108_000316 178
MC_C_20231108_000317 178
M

In [64]:
ref=list(registry.queryDatasets('raw',where=f'exposure.obs_id=\'MC_C_20231108_000313\''))[0]

In [68]:
from astropy.io import fits
fsspec_kwargs = {
    "default_block_size": 100000,
    "default_cache_type": "bytes",
    "endpoint_url": "https://s3dfrgw.slac.stanford.edu"
}
fh=fits.open(butler.getURI(ref).geturl(),use_fsspec=True,
              fsspec_kwargs = fsspec_kwargs)
fh[0].header

SIMPLE  =                    T / Java FITS: Mon Nov 06 17:26:59 PST 2023        
BITPIX  =                    8 / bits per data value                            
NAXIS   =                    0 / number of axes                                 
EXTEND  =                    T / Extensions are permitted                       
DATE    = '2023-11-08T17:58:28.500' / Creation Date and Time of File            
MJD     =   60256.748940972146 / Modified Julian Date that the file was written 
RUNNUM  = '13521   '           / The Run Number                                 
CCD_MANU= 'E2V     '           / CCD Manufacturer                               
CCD_TYPE= 'CCD250  '           / CCD Model Number                               
TESTTYPE= 'BOT_PERSISTENCE' / BIAS, DARK, FE55, FLAT, LAMBDA, PERSISTENCE, SPOT,
IMGTYPE = 'FLAT    '           / BIAS, DARK, FE55, FLAT, FLAT_<lam>, SPOT, PPUMP
DETSIZE = '[1:4096,1:4004]'                                                     
EXPTIME =                 15

In [70]:
len(list(registry.queryDatasets('raw',where=f'exposure.obs_id=\'MC_C_20231107_000675\'')))

202